# History Quiz

## Write a simple class with a few methods for:

- Writing a historical question that has a date as the correct answer
- Getting the correct answer from the LLM
- Getting a human user's best guess at the correct answer
- Checking/reporting the difference between the correct answer and the user's answer

## Loading the libraries

In [ ]:
from datetime import datetime
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI

## Defining the class

In [ ]:
class HistoryQuiz():
    
    def create_history_question(self, topic):
        # this method should output a historical question about the topic that has a date as the correct answer
        # such as "on what date did world war 2 end?"
        system_template = "Create a historical question about the topic {topic} "
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        system_message_prompt.input_variables
        
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])
        chat_prompt.input_variables
        
        prompt = chat_prompt.format_prompt(topic=topic).to_messages()
        
        question = chat(prompt)
        
        return question
    
    def get_AI_answer(self, question):
        
        # This method should get the answer to the historical question from the method above, in datetime format:
        # datetime.datetime(1945, 9, 2, 0, 0)
        
        return correct_datetime
    
    def get_user_answer(self, question):
        
        year = int(input("Enter year: "))
        month = int(input("Enter month (1-12): "))
        day = int(input("Enter day (1-31): "))
        
        user_datetime = datetime(year, month, day)
        
        try:
            return user_datetime
        except ValueError as e:
            return str(e)
    
    
    def check_user_answer(self, user_answer, ai_answer):
        
                # This method should check the user answer against the AI answer and return the difference betwene them
        return answer_difference
        

## Example usage:

In [ ]:
quiz_bot = HistoryQuiz()

### create the question

In [ ]:
question = quiz_bot.create_history_question(topic="World War 2")

In [ ]:
question

### Create the AI answer

In [ ]:
ai_answer = quiz_bot.get_AI_answer(question)

In [ ]:
ai_answer

### Get the user answer

In [ ]:
user_answer = quiz_bot.get_user_answer(question)

In [ ]:
user_answer

### Check the user answer

In [ ]:
quiz_bot.check_user_ansewer(user_answer, ai_answer)